In [1]:
import numpy as np
import pandas as pd 
import matplotlib 
import matplotlib.pyplot as plt


df = pd.read_csv("train.csv")
print(df.head())

        id                                               text author
0  id26305  This process, however, afforded me no means of...    EAP
1  id17569  It never once occurred to me that the fumbling...    HPL
2  id11008  In his left hand was a gold snuff box, from wh...    EAP
3  id27763  How lovely is spring As we looked from Windsor...    MWS
4  id12958  Finding nothing else, not even gold, the Super...    HPL


Writting syles differ in many ways but one such way is that some writers are wordier than others, and some use more or less punctuation than others. This seems especially relevant given that the writers being analysed use several different types of writing (Poe as a poet, Shelly as short stories, Lovecraft as a novelist). There is definitely cross over with these styles but I would argue that the stylistic aspects carry over throughout works so the following functions are designed to assess the writing sections for wordiness and punctuator usage.

In [2]:
import string


def get_word_counts(text):
    count = 0
    for i in text:
        if i == " ":
            count += 1
    return count


def get_punctuator_counts(text):
    punct_dict = {}
    for i in string.punctuation:
        p_count = text.count(i)
        punct_dict[i] = p_count
    return punct_dict


def get_punctuator_array(text):
    punctuator_array_TF = []
    for i in string.punctuation:
        if i in text: 
            punctuator_array_TF.append(1)
        else:
            punctuator_array_TF.append(0)
    return punctuator_array_TF

df["word count"] = df.text.apply(get_word_counts)
df["punctuator_count"] = df.text.apply(get_punctuator_counts)
df["punctuator_array"] = df.text.apply(get_punctuator_array)
df["temp1"] = df.punctuator_count.apply(lambda x: sum(x.values()))
df["temp2"] = df.punctuator_array.apply(lambda x: sum(x))

print(df.head())
print(df.groupby("author", as_index=False)["word count"].mean())
print(df.groupby("author", as_index=False)["temp1"].mean())
print(df.groupby("author", as_index=False)["temp2"].mean())

        id                                               text author  \
0  id26305  This process, however, afforded me no means of...    EAP   
1  id17569  It never once occurred to me that the fumbling...    HPL   
2  id11008  In his left hand was a gold snuff box, from wh...    EAP   
3  id27763  How lovely is spring As we looked from Windsor...    MWS   
4  id12958  Finding nothing else, not even gold, the Super...    HPL   

   word count                                   punctuator_count  \
0          40  {'!': 0, '"': 0, '#': 0, '$': 0, '%': 0, '&': ...   
1          13  {'!': 0, '"': 0, '#': 0, '$': 0, '%': 0, '&': ...   
2          35  {'!': 0, '"': 0, '#': 0, '$': 0, '%': 0, '&': ...   
3          33  {'!': 0, '"': 0, '#': 0, '$': 0, '%': 0, '&': ...   
4          26  {'!': 0, '"': 0, '#': 0, '$': 0, '%': 0, '&': ...   

                                    punctuator_array  temp1  temp2  
0  [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, ...      7      3  
1  [0, 0, 0, 0, 0, 0

The data thus far does not look all that interesting but it does show that Poe tends to use slightly more punctuators and have fewer words (which makes sense for a poet), but also shows that HPL tends to use the least punctuation (somewhat surprising). It also reveals that wordcount/puntuation used might be a good metric to include. Additionally, another set of features that would be interesting to explore are metrics realated to readability. Here, this is looked at in brief with python's readability package. 

In [3]:
import readability


def get_readability_nums(textstr):
    pop_list = ['sentences_per_paragraph', 'type_token_ratio', 'sentences', 'paragraphs']
    res = dict(readability.getmeasures(text=textstr, merge=True))
    for i in pop_list:
        del res[i]
    return res

df["readability"] = df.text.apply(get_readability_nums)
keyslist = list(df.at[0, "readability"].keys())
for i in keyslist:
    df[i] = df.readability.apply(lambda x: x[i])
    print(df.groupby("author", as_index=False)[i].mean())

  author    Kincaid
0    EAP  11.345821
1    HPL  11.949075
2    MWS  11.641965
  author        ARI
0    EAP  12.290581
1    HPL  13.636678
2    MWS  13.181049
  author  Coleman-Liau
0    EAP      8.503033
1    HPL      9.137809
2    MWS      8.636868
  author  FleschReadingEase
0    EAP          59.146649
1    HPL          59.136985
2    MWS          60.534699
  author  GunningFogIndex
0    EAP        15.320904
1    HPL        15.412072
2    MWS        15.698662
  author        LIX
0    EAP  45.796967
1    HPL  47.179099
2    MWS  47.241839
  author  SMOGIndex
0    EAP  11.618737
1    HPL  11.331048
2    MWS  11.659334
  author       RIX
0    EAP  5.199747
1    HPL  5.483230
2    MWS  5.328756
  author  DaleChallIndex
0    EAP        8.990835
1    HPL        9.446937
2    MWS        9.047288
  author  characters_per_word
0    EAP             4.451775
1    HPL             4.480366
2    MWS             4.431556
  author  syll_per_word
0    EAP       1.439720
1    HPL       1.410766
2   

and now i have too many features